In [ ]:
from __future__ import annotations

import os
from pathlib import Path
from datetime import datetime

ROOT = Path.cwd().resolve()
if ROOT.name == "notebooks_v2":
    os.chdir(ROOT.parent)
ROOT = Path.cwd().resolve()
assert (ROOT / "src").exists(), f"Not at repo root. CWD={ROOT}"
print("✅ CWD =", ROOT)

def backup_write(rel_path: str, content: str) -> None:
    p = ROOT / rel_path
    p.parent.mkdir(parents=True, exist_ok=True)
    if p.exists():
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        bak = p.with_suffix(p.suffix + f".bak_{ts}")
        bak.write_text(p.read_text(encoding="utf-8"), encoding="utf-8")
        print("BACKUP:", bak)
    p.write_text(content, encoding="utf-8")
    print("WROTE:", p)

# ---------- PATCH: src/careeragent/api/run_manager_service.py ----------
backup_write("src/careeragent/api/run_manager_service.py", r'''
from __future__ import annotations

import asyncio
import threading
from datetime import datetime, timezone
from pathlib import Path
from typing import Any, Dict, Optional
from uuid import uuid4

from careeragent.services.db_service import SqliteStateStore
from careeragent.langgraph.runtime_nodes import (
    run_single_layer,
    approve_ranking_flow,
    approve_drafts_flow,
)

try:
    from careeragent.config import artifacts_root  # type: ignore
except Exception:
    def artifacts_root() -> Path:
        return Path("src/careeragent/artifacts").resolve()


def utc_now() -> str:
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")


class RunManagerService:
    """
    Description: Background run manager that persists state after each layer for real-time UI.
    Layer: L8
    """

    def __init__(self) -> None:
        self._store = SqliteStateStore()
        self._lock = threading.Lock()

    def _runs_dir(self, run_id: str) -> Path:
        d = artifacts_root() / "runs" / run_id
        d.mkdir(parents=True, exist_ok=True)
        return d

    def save_state(self, *, run_id: str, state: Dict[str, Any]) -> None:
        state.setdefault("meta", {})
        state["meta"]["heartbeat_utc"] = utc_now()
        self._store.upsert_state(
            run_id=run_id,
            status=str(state.get("status", "unknown")),
            state=state,
            updated_at_utc=utc_now(),
        )

    def get_state(self, run_id: str) -> Optional[Dict[str, Any]]:
        return self._store.get_state(run_id=run_id)

    def create_run(self, *, resume_filename: str, resume_text: str, resume_bytes: bytes, preferences: Dict[str, Any]) -> Dict[str, Any]:
        run_id = uuid4().hex
        run_dir = self._runs_dir(run_id)

        (run_dir / "resume_upload.bin").write_bytes(resume_bytes)
        (run_dir / "resume_raw.txt").write_text(resume_text, encoding="utf-8")

        thresholds = (preferences.get("thresholds") or {})
        if "default" in thresholds:
            d = float(thresholds["default"])
            thresholds.setdefault("parser", d)
            thresholds.setdefault("discovery", d)
            thresholds.setdefault("match", d)
            thresholds.setdefault("draft", d)

        state: Dict[str, Any] = {
            "run_id": run_id,
            "status": "running",
            "pending_action": None,
            "hitl_reason": None,
            "hitl_payload": {},
            "thresholds": thresholds,
            "max_retries": int(preferences.get("max_refinements", 3)),
            "layer_retry_count": {},
            "preferences": preferences,
            "resume_filename": resume_filename,
            "resume_text": resume_text,
            "profile": {},
            "jobs_raw": [],
            "jobs_scraped": [],
            "jobs_scored": [],
            "ranking": [],
            "drafts": {},
            "bridge_docs": {},
            "meta": {"created_at_utc": utc_now(), "heartbeat_utc": utc_now(), "last_layer": None},
            "steps": [],  # ✅ progress bar uses this
            "live_feed": [{"layer": "L1", "agent": "API", "message": "Run created. Starting background pipeline…"}],
            "attempts": [],
            "gates": [],
            "evaluations": [],
            "artifacts": {
                "resume_raw": {"path": str(run_dir / "resume_raw.txt"), "content_type": "text/plain"},
                "resume_upload": {"path": str(run_dir / "resume_upload.bin"), "content_type": "application/octet-stream"},
            },
        }

        self.save_state(run_id=run_id, state=state)
        return state

    def start_background(self, run_id: str) -> None:
        t = threading.Thread(target=self._bg_runner, args=(run_id,), daemon=True)
        t.start()

    def _bg_runner(self, run_id: str) -> None:
        asyncio.run(self._bg_async(run_id))

    async def _run_layer(self, state: Dict[str, Any], layer: str, timeout_sec: int) -> Dict[str, Any]:
        try:
            return await asyncio.wait_for(run_single_layer(state, layer), timeout=timeout_sec)
        except asyncio.TimeoutError:
            state["status"] = "needs_human_approval"
            state["pending_action"] = f"timeout_{layer.lower()}"
            state.setdefault("live_feed", []).append({"layer": layer, "agent": "TimeoutGuard", "message": f"{layer} timed out after {timeout_sec}s"})
            return state
        except Exception as e:
            state["status"] = "failed"
            state["pending_action"] = f"error_{layer.lower()}"
            state.setdefault("live_feed", []).append({"layer": layer, "agent": "CrashGuard", "message": f"{layer} crashed: {e}"})
            return state

    async def _bg_async(self, run_id: str) -> None:
        state = self.get_state(run_id)
        if not state:
            return

        # IMPORTANT: persist after each layer so UI updates
        plan = [("L0", 45), ("L2", 60), ("L3", 120), ("L4", 180), ("L5", 45)]

        for layer, tmo in plan:
            if state.get("status") in ("blocked", "needs_human_approval", "failed", "completed"):
                self.save_state(run_id=run_id, state=state)
                return

            state.setdefault("meta", {})
            state["meta"]["last_layer"] = layer

            state.setdefault("steps", []).append({"layer_id": layer, "status": "running", "started_at_utc": utc_now()})
            state.setdefault("live_feed", []).append({"layer": layer, "agent": "Orchestrator", "message": f"Running {layer}…"})
            self.save_state(run_id=run_id, state=state)

            state = await self._run_layer(state, layer, tmo)

            state["steps"][-1]["status"] = state.get("status", "running")
            state["steps"][-1]["finished_at_utc"] = utc_now()
            self.save_state(run_id=run_id, state=state)

            # Stop for HITL when ranking is ready
            if state.get("pending_action") == "review_ranking":
                state["status"] = "needs_human_approval"
                self.save_state(run_id=run_id, state=state)
                return

        # If still running after L5, force ranking review
        if state.get("status") == "running":
            state["status"] = "needs_human_approval"
            state["pending_action"] = "review_ranking"
            state.setdefault("live_feed", []).append({"layer": "L5", "agent": "Orchestrator", "message": "Ranking ready for review."})
            self.save_state(run_id=run_id, state=state)

    async def execute_layer(self, *, run_id: str, layer: str) -> Dict[str, Any]:
        state = self.get_state(run_id) or {}
        state = await run_single_layer(state, layer.upper())
        self.save_state(run_id=run_id, state=state)
        return state

    async def handle_action(self, *, run_id: str, action_type: str, payload: Dict[str, Any]) -> Dict[str, Any]:
        state = self.get_state(run_id)
        if not state:
            raise ValueError("run_id not found")

        state.setdefault("meta", {})
        state["meta"]["last_user_action"] = {"type": action_type, "payload": payload, "at_utc": utc_now()}

        if action_type == "execute_layer":
            layer = str(payload.get("layer", "")).upper()
            state = await run_single_layer(state, layer)
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "resume_cleanup_submit":
            txt = str(payload.get("resume_text", "")).strip()
            if txt:
                run_dir = self._runs_dir(run_id)
                (run_dir / "resume_raw.txt").write_text(txt, encoding="utf-8")
                state["resume_text"] = txt
                state["artifacts"]["resume_raw"] = {"path": str(run_dir / "resume_raw.txt"), "content_type": "text/plain"}
                state["status"] = "running"
                state["pending_action"] = None
                state.setdefault("live_feed", []).append({"layer":"L2","agent":"HITL","message":"Resume updated. Restarting pipeline…"})
                self.save_state(run_id=run_id, state=state)
                self.start_background(run_id)
            return state

        if action_type == "approve_ranking":
            state = await approve_ranking_flow(state)
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "reject_ranking":
            reason = str(payload.get("reason", "")).strip()
            state.setdefault("meta", {})
            state["meta"].setdefault("refinement_notes", [])
            if reason:
                state["meta"]["refinement_notes"].append(reason)
            state["status"] = "running"
            state["pending_action"] = None
            state.setdefault("live_feed", []).append({"layer":"L5","agent":"HITL","message":"Ranking rejected. Re-running pipeline…"})
            self.save_state(run_id=run_id, state=state)
            self.start_background(run_id)
            return state

        if action_type == "approve_drafts":
            state = await approve_drafts_flow(state)
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "reject_drafts":
            state["status"] = "needs_human_approval"
            state["pending_action"] = "review_ranking"
            state.setdefault("live_feed", []).append({"layer":"L6","agent":"HITL","message":"Drafts rejected. Back to ranking."})
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type in ("approve_job", "reject_job"):
            jid = str(payload.get("job_id",""))
            state.setdefault("meta", {})
            state["meta"].setdefault("approved_jobs", [])
            state["meta"].setdefault("rejected_jobs", [])
            if action_type == "approve_job":
                state["meta"]["approved_jobs"].append({"job_id": jid, "at_utc": utc_now()})
                state.setdefault("live_feed", []).append({"layer":"L5","agent":"HITL","message":f"Approved job {jid[:24]}…"})
            else:
                state["meta"]["rejected_jobs"].append({"job_id": jid, "at_utc": utc_now()})
                state.setdefault("live_feed", []).append({"layer":"L5","agent":"HITL","message":f"Rejected job {jid[:24]}…"})
            self.save_state(run_id=run_id, state=state)
            return state

        state.setdefault("live_feed", []).append({"layer":"L5","agent":"HITL","message":f"Unhandled action_type={action_type}"})
        self.save_state(run_id=run_id, state=state)
        return state
''')

# ---------- PATCH: src/careeragent/api/main.py ----------
backup_write("src/careeragent/api/main.py", r'''
from __future__ import annotations

import json
from typing import Any, Dict

from fastapi import FastAPI, File, Form, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware

from careeragent.api.request_models import AnalyzeResponse, ActionRequest
from careeragent.api.run_manager_service import RunManagerService


app = FastAPI(title="CareerAgent-AI Brain", version="0.3.0")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

RM = RunManagerService()


@app.get("/health")
def health() -> Dict[str, Any]:
    return {"status": "ok"}


def _extract_text(filename: str, data: bytes) -> str:
    name = (filename or "").lower()
    if name.endswith(".txt"):
        return data.decode("utf-8", errors="replace")
    if name.endswith(".pdf"):
        from pypdf import PdfReader  # type: ignore
        import io
        reader = PdfReader(io.BytesIO(data))
        return "\n".join([(pg.extract_text() or "") for pg in reader.pages])
    if name.endswith(".docx"):
        import docx  # type: ignore
        import io
        d = docx.Document(io.BytesIO(data))
        return "\n".join([p.text for p in d.paragraphs if p.text])
    return data.decode("utf-8", errors="replace")


@app.post("/analyze", response_model=AnalyzeResponse)
async def analyze(resume: UploadFile = File(...), preferences_json: str = Form(...)) -> AnalyzeResponse:
    try:
        prefs = json.loads(preferences_json)
    except Exception:
        raise HTTPException(status_code=400, detail="Invalid preferences_json")

    data = await resume.read()
    text = _extract_text(resume.filename or "resume.txt", data)

    st = RM.create_run(resume_filename=resume.filename or "resume.txt", resume_text=text, resume_bytes=data, preferences=prefs)
    RM.start_background(st["run_id"])
    return AnalyzeResponse(run_id=st["run_id"], status=st["status"])


@app.get("/status/{run_id}")
def status(run_id: str) -> Dict[str, Any]:
    st = RM.get_state(run_id)
    if not st:
        raise HTTPException(status_code=404, detail="run_id not found")
    return st


@app.post("/action/{run_id}")
async def action(run_id: str, req: ActionRequest) -> Dict[str, Any]:
    try:
        return await RM.handle_action(run_id=run_id, action_type=req.action_type, payload=req.payload)
    except ValueError:
        raise HTTPException(status_code=404, detail="run_id not found")
''')

print("✅ Patched backend to stream state updates per layer (real-time progress).")
print("Restart backend WITHOUT --reload for long runs.")

In [ ]:
RUN_ID=da8aa45e4eec4ada8dae9a77b95c6642
curl -sS "http://127.0.0.1:8000/status/$RUN_ID" | python -c "import sys,json; s=json.load(sys.stdin); print('status',s.get('status'),'pending',s.get('pending_action')); print('last_layer',(s.get('meta') or {}).get('last_layer'),'steps',len(s.get('steps') or []),'feed',len(s.get('live_feed') or [])); print('jobs_raw',len(s.get('jobs_raw') or []),'jobs_scored',len(s.get('jobs_scored') or []),'ranking',len(s.get('ranking') or []))"